In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

sys.path.append(os.path.join(os.getcwd()[:os.getcwd().find('notebooks')]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import string
import numpy as np
import pandas as pd
from collections import defaultdict

from config import cfg

base_path = os.path.join(os.getcwd()[:os.getcwd().find('notebooks')])

### Pairs

In [96]:
pairs = pd.read_excel(os.path.join(cfg["docs_path"], "groups", "Bld Pairs.xlsx"), index_col=0)

pairs_long = (
    pairs
    .melt(ignore_index=False)
    .reset_index()
    .dropna()
    .assign(letter_pair=lambda df: df["index"] + df["variable"])
    .rename(columns={"value": "image"})
    .pipe(lambda df: df[["letter_pair", "image"]])
    .pipe(lambda df: df[df.image != "."])
    # .to_csv(os.path.join(cfg["docs_path"], "bld_pairs_long.csv"))
)

### Split in groups

In [89]:
learn_last = lambda x: ("A" in x) or ("E" in x) or ("R" in x)

pair_groups = (
    pairs_long
    .assign(learn_last = lambda df: df["letter_pair"].map(learn_last))
    .assign(rank=lambda df: df["letter_pair"].rank())
    .assign(rank=lambda df: df.apply(lambda x: x["rank"] * 100 if x.learn_last else x["rank"], axis=1))
    .assign(rank=lambda df: df["rank"].rank())
    .sort_values("rank")
    .assign(first_letter=lambda df: df["letter_pair"].map(lambda x: x[0]))
)

main_groups = pair_groups.pipe(lambda df: df[~df.learn_last])
last_groups = pair_groups.pipe(lambda df: df[df.learn_last])

In [90]:
for l, group in main_groups.groupby("first_letter"):
    group[["letter_pair", "image"]].to_csv(os.path.join(cfg["docs_path"], "groups", f"bld_pairs_{l}.csv"), index=False)

last_groups[["letter_pair", "image"]].to_csv(os.path.join(cfg["docs_path"], "groups", f"bld_pairs_Z.csv"), index=False)